# Notebook to calculate sarcopenia area for MRI

## Import Libraries and Set directories

In [1]:
#Import Libraries
import os
import SimpleITK as sitk
import pandas as pd
import numpy as np
import re
import matplotlib.figure as figure

from myshow import *

In [2]:
# Set input/output directories
cwd = os.getcwd()
dir_nifti ='/home/jovyan/data'
dir_output = '/home/jovyan/output'
print(dir_nifti)
print(dir_output)

/home/jovyan/data
/home/jovyan/output


## Define some handy functions for creating dicom 

In [3]:
# load nifti list
patslist = os.listdir(dir_nifti)
pats = [pat for pat in patslist if 'xls' not in pat]
in_file = [pat for pat in patslist if 'xls' in pat][0]
print('No of patient folders to process: ', len(pats))
print('Input Excel file: ', in_file)

No of patient folders to process:  201
Input Excel file:  sarcopenia data_ES_043021_area_calculation_byLisa.xls


In [4]:
def get_mask_paths(patpath):
    for root, directories, files in os.walk(patpath):
        for filename in files:
            # join the two strings in order to form the full filepath.
            if 'mask' in filename:
                filepath = os.path.join(root, filename)
                return filepath
            
        print('Nifti mask not found for this patient:', patpath)    
        break

def get_axialmr_paths(patpath):
    for root, directories, files in os.walk(patpath):
        for filename in files:
            # join the two strings in order to form the full filepath.
            if 'Ax' in filename:
                filepath = os.path.join(root, filename)
                return filepath
            
        print('Axial Nifti not foudn for patient:', patpath)    
        break

In [13]:
# New 
def write_L3_mrandmask(path,pat,out=dir_output):
    # Set paths
    outpath = os.path.join(dir_output,pat)
    os.makedirs(outpath,exist_ok=True)
    inpath = os.path.join(path,pat)
    
    ## Get the 2D mask slice
    maskfile = get_mask_paths(inpath)
    img = sitk.ReadImage(maskfile)

    img_arr = sitk.GetArrayFromImage(img)
    mask_slices = np.unique(np.nonzero(img_arr)[0])
    if len(mask_slices) > 1:
        print('More than 1 axial slice has non zero pixels for pat: ', pat)
    z_loc = mask_slices[0]
    img_2darr = img_arr[z_loc,:,:]
    # Change all labels to skeletal muscle for segmenting both skeletal and psoas together
    img_2darr[img_2darr > 1] = 1
    mask_img = sitk.GetImageFromArray(img_2darr)
    mask_img.SetSpacing(img.GetSpacing()[:-1])
    
    ## Get 2D Axial MR slice at L3
    mrfile = get_axialmr_paths(inpath)
    mrimg = sitk.ReadImage(mrfile)
    mrimg_arr = sitk.GetArrayFromImage(mrimg)
    l3_arr= mrimg_arr[z_loc,:,:]
    l3_arr = l3_arr.astype(np.int16)
    l3_img = sitk.GetImageFromArray(l3_arr)
    l3_img.SetSpacing(mrimg.GetSpacing()[:-1])
    
    ## Resize to 256x256
    new_size = [256,256]
    # Set reference image for resampling
    reference_image = sitk.Image(new_size, l3_img.GetPixelIDValue())
    reference_image.SetOrigin(l3_img.GetOrigin())
    reference_image.SetDirection(l3_img.GetDirection())
    reference_image.SetSpacing([sz*spc/nsz for nsz,sz,spc in zip(new_size, l3_img.GetSize(), mrimg.GetSpacing()[:-1])])    
    #l3_img_smooth = sitk.Resample(sitk.SmoothingRecursiveGaussian(l3_img1[:,:,]), reference_image)
    
    orig_spacing = mrimg.GetSpacing()[0]
    orig_size = l3_img.GetSize()[0]
    l3_img_smooth = sitk.Resample(l3_img, reference_image)
    mask_img_smooth = sitk.Resample(mask_img,reference_image,sitk.TranslationTransform(l3_img.GetDimension()),sitk.sitkNearestNeighbor)
    
    
    # Book Keeping
    orig_spacing = mrimg.GetSpacing()[0]
    orig_size = l3_img.GetSize()[0]
    new_spacing = l3_img_smooth.GetSpacing()[0]
    new_size = l3_img_smooth.GetSize()[0]
    
    ## Write TIF images as output
    print('Pixel Type: ',l3_img_smooth.GetPixelIDTypeAsString())
    sitk.WriteImage(l3_img_smooth, os.path.join(outpath,'MR.tif') )
    sitk.WriteImage(mask_img_smooth, os.path.join(outpath,'MASK.tif') )
    
    return orig_size, orig_spacing, new_size, new_spacing, z_loc

In [14]:
df = pd.DataFrame(columns = ['pats','slice_no','orig_size','orig_spacing','orig_pixelarea','new_size','new_spacing','new_pixelarea'])
for i,pat in enumerate(reversed(pats)):
    print('processing pat: ',pat)
    orig_size,orig_spacing,new_size,new_spacing,z_loc = write_L3_mrandmask(dir_nifti,pat)
    df.loc[len(df)] = [pat,z_loc,orig_size,orig_spacing,orig_spacing**2,new_size,new_spacing,new_spacing**2]


display(df.head(10))
df.to_csv(os.path.join(dir_output,'image_statistics_resampling.csv'),index=False)
print('all patient processed')

processing pat:  exam 9b
Pixel Type:  16-bit signed integer
processing pat:  exam 99
Pixel Type:  16-bit signed integer
processing pat:  exam 98
Pixel Type:  16-bit signed integer
processing pat:  exam 97b
Pixel Type:  16-bit signed integer
processing pat:  exam 96b
Pixel Type:  16-bit signed integer
processing pat:  exam 95b
Pixel Type:  16-bit signed integer
processing pat:  exam 94b
Pixel Type:  16-bit signed integer
processing pat:  exam 93
Pixel Type:  16-bit signed integer
processing pat:  exam 92
Pixel Type:  16-bit signed integer
processing pat:  exam 91
Pixel Type:  16-bit signed integer
processing pat:  exam 90
Pixel Type:  16-bit signed integer
processing pat:  exam 89c
Pixel Type:  16-bit signed integer
processing pat:  exam 88
Pixel Type:  16-bit signed integer
processing pat:  exam 87
Pixel Type:  16-bit signed integer
processing pat:  exam 86b
Pixel Type:  16-bit signed integer
processing pat:  exam 85
Pixel Type:  16-bit signed integer
processing pat:  exam 84
Pixel Typ

Pixel Type:  16-bit signed integer
processing pat:  exam 157
Pixel Type:  16-bit signed integer
processing pat:  exam 156b
Pixel Type:  16-bit signed integer
processing pat:  exam 155
Pixel Type:  16-bit signed integer
processing pat:  exam 154
Pixel Type:  16-bit signed integer
processing pat:  exam 153
Pixel Type:  16-bit signed integer
processing pat:  exam 152b
Pixel Type:  16-bit signed integer
processing pat:  exam 151
Pixel Type:  16-bit signed integer
processing pat:  exam 150b
Pixel Type:  16-bit signed integer
processing pat:  exam 15
Pixel Type:  16-bit signed integer
processing pat:  exam 148b
Pixel Type:  16-bit signed integer
processing pat:  exam 147b
Pixel Type:  16-bit signed integer
processing pat:  exam 146
Pixel Type:  16-bit signed integer
processing pat:  exam 145
Pixel Type:  16-bit signed integer
processing pat:  exam 144b
Pixel Type:  16-bit signed integer
processing pat:  exam 143
Pixel Type:  16-bit signed integer
processing pat:  exam 142
Pixel Type:  16-bit

,pats,slice_no,orig_size,orig_spacing,orig_pixelarea,new_size,new_spacing,new_pixelarea
0,exam 9b,36,512,0.781300,0.610430,256,1.562600,2.441719
1,exam 99,10,512,0.664100,0.441029,256,1.328200,1.764115
2,exam 98,45,320,0.781250,0.610352,256,0.976562,0.953674
3,exam 97b,34,512,0.546900,0.299100,256,1.093800,1.196398
4,exam 96b,38,512,0.546900,0.299100,256,1.093800,1.196398
5,exam 95b,40,512,0.585900,0.343279,256,1.171800,1.373115
6,exam 94b,56,352,0.909091,0.826446,256,1.250000,1.562500
7,exam 93,43,320,0.812500,0.660156,256,1.015625,1.031494
8,exam 92,16,512,0.703100,0.494350,256,1.406200,1.977399
9,exam 91,38,384,0.781250,0.610352,256,1.171875,1.373291


all patient processed
